# Análise Comparativa de Modelos

Nesse notebook iremos olhar para os textos obtidos na primeira unidade para construir modelos de aprendizado de máquina. Para isso, precisaremos de algumas bibliotecas já conhecidas (*pandas, nltk, numpy*) e de outras novas, relacionadas à:
- preparação dos dados: 
    - *CountVectorizer*, 
    - *TfidfVectorizer*,
    - *TruncatedSVD*
    - *StandardScaler*
- aprendizagem em si: 
    - *LogisticRegression*, 
    - *KNeighborsClassifier*, 
    - *SVC*, 
    - *RandomForestClassifier*
- construção da sequência de passos a realizar:
    - *Pipeline*
- avaliação e seleção dos modelos obtidos:
    - *ShuffleSplit*,
    - *cross_validate*
    
Algumas dessas funções serão explicadas adiante. Sigamos.

## Importando bibliotecas

In [ ]:
from itertools import product # produto cartesiano de duas listas sem precisar de for aninhados
import pandas as pd # manipulação de dataframes
import nltk # ferramentas p/ processamento de linguagem natural
import numpy as np # manipulação de matrizes, funções matemáticas
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import ShuffleSplit, cross_validate

nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /home/madson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## Carregando Dados

In [2]:
# Adquire dados do arquivo produzido na etapa anterior
input_path = "../data/interim/news.csv"
df = pd.read_csv(input_path)

# Converte as colunas com texto em uma lista
corpus = df.text.to_list()

# Atribui os valores 1 para notícias verdadeiras e 0 para notícias falsas
labels = df.label.replace({"true": 1, "fake": 0})

## Conversões dos textos

### CountVectorizer

Esse método de conversão cria uma matriz com a contagem de todas as palavras presentes em todos os textos considerados. Esse método também é chamado de Bag of Words, ou Saco de Palavras.

![CountVectorizer](images/count-vectorizer.png)

### TfidfVectorizer

Esse método de conversão cria uma matriz que considera dois valores: TF e DF. TF ($tf_{i,j}$) é a contagem de uma palavra ($i$) no texto ($j$), tal como é calculada pelo CountVectorizer. Já DF ($df_{i}$) diz respeito à quantidade de textos em que aquela palavra aparece. No cálculo, o valor para uma dada palavra ($w_{i,j}$) é dado por:

$ w_{i,j} = tf_{i,j} \times log(\frac{N}{df_{i}}) $

Em que $N$ é o número total de documentos. Como $df_{i}$ está no denominador do logarítmo, trata-se de uma conversão que considera TF e o inverso de DF, iDF.

![CountVectorizer](images/tfidf-vectorizer.png)

Em ambos os casos, palavras comuns e sem importância semântica no idioma trabalhado devem ser removidas. Afinal, caso fossem consideradas na conversão TFiDF, as palavras "de" e "desse" no primeiro texto teriam o mesmo peso da palavra "corrupção" e "suspeito", com importância semântica superior.

Aqui, optamos também por utilizar apenas as 1000 palavras mais frequentes em ambas as conversões, de modo a limitar o número de características para o treinamento.

In [4]:
vectorizers = {
    'bow': CountVectorizer(
        stop_words = nltk.corpus.stopwords.words('portuguese'),
        max_features = 1000
    ),
    'tfidf': TfidfVectorizer(
        stop_words = nltk.corpus.stopwords.words('portuguese'),
        max_features = 1000
    )
}

## Modelos de Aprendizagem de Máquina

### Regressão Logística

A regressão logística, que é um modelo classificador, utiliza uma função logística para calcular as probabilidades de uma notícia ser falsa ou verdadeira. A função logística por trás deste modelo é uma curva sigmoide contendo as probabilidades de classificação em uma determinada categoria (notícia falsa ou verdadeira). A partir disso se encontra um valor limite dentro da curva para que então se obtenham as predições.

### KNN

Aqui a classificação das notícias em falsas ou verdadeira é feita com base nos K-vizinhos mais próximos (*K-Nearest Neighbors*) a ela. Se utilizamos $K = 10$ e dentre as 10 notícias mais próximas de uma notícia $i$ no nosso espaço dimensional, 6 são falsas e 4 são verdadeiras, esta notícia $i$ será classificada como falsa. No nosso caso utilizamos $K = 21$.

### SVC

O SVC é um *Support Vector Classifier*, isso é, um algoritmo de classificação que utiliza vetores (dados) como suporte para definir um hiperplano que separe dados de um grupo dos dados de outro grupo. No nosso caso, o SVC busca um hiperplano que separe as matrizes produzidas a partir de notícias falsas daquelas de notícias verdadeiras.

O hiperplano pode ser obtido após uma transformação dos dados, na qual eles saem de seu espaço dimensional original, intrínseco, e são levados a um espaço dimensional superior. Essa transformação é feita através através de diversas fórmulas matemáticas para que então o hiperplano ideal - aquele que separa os dados com melhor margem - seja identificado; este é o chamado *kernel trick*.

### Random Forest



In [5]:
models = {
    'logistic regression': LogisticRegression(),
    'KNN': KNeighborsClassifier(n_neighbors=21),
    'SVC': SVC(),
    'random forest': RandomForestClassifier()
}

In [6]:
pca = TruncatedSVD(500)
scaler = StandardScaler()

In [7]:
results = {}
n_splits = 10
split = ShuffleSplit(n_splits=n_splits, test_size=.2)
for vectorizer, model in product(vectorizers.items(), models.items()):
    vectorizer_name, vectorizer_ = vectorizer
    model_name, model_ = model
    pipeline = Pipeline(steps=[
        ("vectorizer", vectorizer_),
        ("pca", pca),
        ("normalize", scaler),
        ("model", model_)
    ])    
    scores = cross_validate(pipeline, corpus, labels, cv=split, scoring=['accuracy', 'f1'])
    scores['model'] = [f"{vectorizer_name}-{model_name}"] * n_splits
    if not(results):
        results = {key: [] for key in scores}
    for key in scores:
        results[key].extend(scores[key])

In [8]:
df_results = (
    pd
    .DataFrame(results)
    .groupby("model")
    .agg([np.mean, np.std])
    .transpose()
)

In [9]:
df_results

model                bow-KNN   bow-SVC  bow-logistic regression  \
fit_time      mean  5.814648  8.637069                 6.292938   
              std   0.080802  0.084062                 0.187033   
score_time    mean  0.924707  1.093691                 0.654466   
              std   0.026855  0.047469                 0.023749   
test_accuracy mean  0.507500  0.952847                 0.952292   
              std   0.015136  0.003456                 0.005689   
test_f1       mean  0.010868  0.952411                 0.951610   
              std   0.004602  0.003727                 0.006216   

model               bow-random forest  tfidf-KNN  tfidf-SVC  \
fit_time      mean          21.823260   5.813607  14.061045   
              std            0.333607   0.073308   0.339682   
score_time    mean           0.612344   0.920975   3.191312   
              std            0.010742   0.037061   0.079363   
test_accuracy mean           0.946319   0.500278   0.948194   
              std            0.005947   0.013797   0.003092   
test_f1       mean           0.947615   0.661847   0.949383   
              std            0.005971   0.012097   0.002667   

model               tfidf-logistic regression  tfidf-random forest  
fit_time      mean                   6.063270            16.788882  
              std                    0.079474             0.287679  
score_time    mean                   0.652346             0.611891  
              std                    0.017939             0.015909  
test_accuracy mean                   0.940486             0.914097  
              std                    0.005498             0.007685  
test_f1       mean                   0.940806             0.912608  
              std                    0.005817             0.007970